In [8]:
import logging
from typing import Annotated

from autogen import register_function
from autogen.agentchat import ConversableAgent
from fixtures import openai_gpt4o_mini_llm_config

from fastagency import UI, IOMessage
from fastagency.ui.console import ConsoleUI
from fastagency.runtimes.autogen import AutoGenWorkflows
from fastagency.logging import get_logger

In [9]:
# Get the logger
logger = get_logger(__name__)

# Log messages
logger.warning("warning message")
logger.info("info message")

[WARNING] warning message
[INFO] info message


In [10]:
wf = AutoGenWorkflows()


@wf.register(name="simple_learning", description="Student and teacher learning chat")
def simple_workflow(wf: AutoGenWorkflows, ui: UI, initial_message: str, session_id: str) -> str:
    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message="You are a student willing to learn.",
        llm_config=openai_gpt4o_mini_llm_config,
    )
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message="You are a math teacher.",
        llm_config=openai_gpt4o_mini_llm_config,
    )

    chat_result = student_agent.initiate_chat(
        teacher_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=5,
    )

    return chat_result.summary

In [11]:
def run_workflow(name: str, initial_message: str):

    io = ConsoleUI()

    ui.process_message(
        IOMessage.create(
            sender="user",
            recipient="workflow",
            type="system_message",
            message={
                "heading": "Workflow BEGIN",
                "body": f"Starting workflow with initial_message: {initial_message}"
            },
        )
    )

    result = wf.run(name=name, session_id="session_id", io=io.create_subconversation(), initial_message=initial_message)

    ui.process_message(
        IOMessage.create(
            sender="user",
            recipient="workflow",
            type="system_message",
            message={
                "heading": "Workflow END",
                "body": f"Ending workflow with result: {result}"
            },
        )
    )


In [12]:
run_workflow("simple_learning", "What is triangle inequality?")

+--------------------------------------------------------------------------------+
|
| user -> workflow: system_message
|
| {
|   "message": {
|     "heading": "Workflow BEGIN",
|     "body": 
| "Starting workflow with initial_message: What is triangle inequality?"
| 
|   }
| }
+--------------------------------------------------------------------------------+

    +--------------------------------------------------------------------------------+
    |
    | Student_Agent -> Teacher_Agent: text_message
    |
    | What is triangle inequality?

    +--------------------------------------------------------------------------------+

    +--------------------------------------------------------------------------------+
    |
    | Teacher_Agent -> Student_Agent: text_message
    |
    | The triangle inequality is a fundamental property of triangles in 
    | geometry that states that, for any triangle, the sum of the lengths of
    |  any two sides must be greater than the length of the thi

In [13]:
# wf = AutoGenWorkflows()

from fastagency.base import MultipleChoice, SystemMessage, TextInput


@wf.register(name="exam_practice", description="Student and teacher chat")
def exam_learning(wf: AutoGenWorkflows, ui: UI, initial_message: str, session_id: str) -> str:

    def is_termination_msg(msg: str) -> bool:
        return msg["content"] is not None and "TERMINATE" in msg["content"]

    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message="You are a student writing a practice test. Your task is as follows:\n"
            "  1) Retrieve exam questions by calling a function.\n"
            "  2) Write a draft of proposed answers and engage in dialogue with your tutor.\n"
            "  3) Once you are done with the dialogue, register the final answers by calling a function.\n"
            "  4) Retrieve the final grade by calling a function.\n"
            "Finally, terminate the chat by saying 'TERMINATE'.",
        llm_config=openai_gpt4o_mini_llm_config,
        human_input_mode="NEVER",
        is_termination_msg=is_termination_msg,
    )
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message="You are a teacher.",
        llm_config=openai_gpt4o_mini_llm_config,
        human_input_mode="NEVER",
        is_termination_msg=is_termination_msg,
    )

    def retrieve_exam_questions(message: Annotated[str, "Message for examiner"]) -> str:
        # return "1) Mona Lisa 2) Innovations 3) Florence at the time of Leonardo 4) The Last Supper 5) Vit"
        try:
            msg = TextInput(
                sender="student",
                recipient="teacher",
                prompt=message,
                suggestions=["1) Mona Lisa", "2) Innovations", "3) Florence at the time of Leonardo", "4) The Last Supper", "5) Vit"],
            )
            return ui.process_message(msg)
        except Exception as e:
            logger.error(f"retrieve_exam_questions() FAILED! {e}")
            return f"retrieve_exam_questions() FAILED! {e}"

    def write_final_answers(message: Annotated[str, "Message for examiner"]) -> str:
        try:
            msg = SystemMessage(
                sender="function call logger",
                recipient="system",
                message={
                    "operation": "storing final answers",
                    "content": message,
                },
            )
            ui.process_message(msg)
            return "Final answers stored."
        except Exception as e:
            logger.error(f"write_final_answers() FAILED! {e}")
            return f"write_final_answers() FAILED! {e}"

    def get_final_grade(message: Annotated[str, "Message for examiner"]) -> str:
        try:
            msg = MultipleChoice(
                    sender="student",
                    recipient="teacher",
                    prompt=message,
                    choices=["A", "B", "C", "D", "F"],
            )
            return ui.process_message(msg)
        except Exception as e:
            logger.error(f"get_final_grade() FAILED! {e}")
            return f"get_final_grade() FAILED! {e}"

    register_function(
        retrieve_exam_questions,
        caller=student_agent,
        executor=teacher_agent,
        name="retrieve_exam_questions",
        description="Get exam questions from examiner",
    )

    register_function(
        write_final_answers,
        caller=student_agent,
        executor=teacher_agent,
        name="write_final_answers",
        description="Write a final answers to exam questions to examiner, but only after discussing with the tutor first.",
    )

    register_function(
        get_final_grade,
        caller=student_agent,
        executor=teacher_agent,
        name="get_final_grade",
        description="Get the final grade after submitting the answers.",
    )

    chat_result = teacher_agent.initiate_chat(
        student_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=10,
    )

    return chat_result.summary

In [14]:
run_workflow("exam_practice", "Prepare for the test about Leonardo da Vinci.")


+--------------------------------------------------------------------------------+
|
| user -> workflow: system_message
|
| {
|   "message": {
|     "heading": "Workflow BEGIN",
|     "body": 
| "Starting workflow with initial_message: Prepare for the test about 
| Leonardo da Vinci."
|   }
| }
+--------------------------------------------------------------------------------+

    +--------------------------------------------------------------------------------+
    |
    | Teacher_Agent -> Student_Agent: text_message
    |
    | Prepare for the test about Leonardo da Vinci.

    +--------------------------------------------------------------------------------+

    +--------------------------------------------------------------------------------+
    |
    | Student_Agent -> Teacher_Agent: suggested_function_call
    |
    | {
    |   "function_name": "retrieve_exam_questions",
    |   "call_id": 
    | "call_WtPX2rq3saIIqk4hM4jjAhNX",
    |   "arguments": {
    |     "message": 
    

1) Vituvian man 2) Mona Lisa 3) Medici family